Do some ppts just answer randomly to traits questions?
Indication of random responses: very little time between responses.
So compute stats on time between responses.

In [1]:
from textrec.paths import paths
import json
import pandas as pd
from textrec import logs_to_csv
import tqdm
from collections import OrderedDict

/Users/kcarnold/anaconda3/envs/py36/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


Loading ONMT models...
coco_lm_adam_acc_46.00_ppl_16.32_e10_nooptim.pt
Loading model parameters.
coco_cap_adam_acc_48.73_ppl_12.56_e10_nooptim.pt
Loading model parameters.
Ready.
Loading SpaCy...done


In [2]:
participants_by_batch = logs_to_csv.get_participants_by_batch()
gc1 = ['gc1']

In [3]:
participants = [(batch, id) for batch in ['gc1', 'spec1', 'spec2'] for id in participants_by_batch[batch]]

In [4]:
all_results = []
for batch, participant in tqdm.tqdm_notebook(participants):
    with open(paths.logdir / f'{participant}.jsonl') as f:
        log = [json.loads(line.strip()) for line in f]
    responses = {}

    for i, event in enumerate(log):
        if event['type'] != 'controlledInputChanged':
            continue
        if isinstance(event['value'], str):
            continue

        if event['name'] in responses:
            prev_delay = responses[event['name']]['delay']
        else:
            prev_delay = 0.
            
        # Find the previous event that was actually a user input (has a 'jsTimestamp').
        prev_idx = i - 1
        while 'jsTimestamp' not in log[prev_idx]:
            prev_idx -= 1
        responses[event['name']] = dict(
            num_updates=0,
            value=event['value'],
            delay=prev_delay + (log[i]['jsTimestamp'] - log[prev_idx]['jsTimestamp']) / 1000)
    
    for name, data in responses.items():
        entry = OrderedDict()
        entry['batch'] = batch
        entry['participant'] = participant
        entry['name'] = name
        entry.update(data)
        all_results.append(entry)
        
    del responses

In [5]:
df = pd.DataFrame(all_results)

In [6]:
df.set_index(['batch', 'participant', 'name']).value.unstack(level=-1).to_csv('survey_responses.csv')

In [7]:
agg = df.groupby(['batch', 'participant']).delay.aggregate(['mean', 'median', 'min'])

In [8]:
agg.rename(columns={col: 'response_time_'+col for col in agg.columns}).to_csv('thoughtfulness.csv')

In [9]:
df.groupby('participant').delay.median().sort_values()

participant
7q253f    0.7000
46gc8v    0.8400
pr5hff    0.8810
w3xwhj    1.0260
w46px5    1.0450
559x69    1.1000
3pcfjp    1.1270
77j4mf    1.1360
gmvv6r    1.1990
fh7mpr    1.2520
9c8wwq    1.2710
4x7jjx    1.2965
2q4327    1.3120
63mrcq    1.3170
ch945r    1.3210
c2wv83    1.3230
9rpwf7    1.3710
6pmv8x    1.4010
cf73g3    1.4060
85c66x    1.4250
phqcw9    1.4340
p45469    1.4360
94wwrq    1.4490
vj2c8r    1.4580
vg926m    1.4880
654p57    1.5150
r453r6    1.5190
f3892v    1.5480
9f5xwx    1.5530
7xcppv    1.5550
           ...  
gvwqp6    2.6570
rp3jwh    2.6650
gg65g6    2.6950
wf82v8    2.8260
533r6c    2.8310
gw3w72    2.8550
vxjcf7    2.8850
26cj6j    2.9070
24ccf8    2.9100
7x6428    2.9125
f8frp2    2.9170
26w4jv    2.9340
qgxp9h    3.0660
7hxmrr    3.0730
p475m7    3.2610
phv4hj    3.2820
wcmr62    3.3230
46qwpm    3.3810
w768j6    3.4220
v6fc3x    3.4760
mh78w9    3.5500
5r26hq    3.6410
5pf4h7    3.6525
3ppjq7    3.6790
7grhw7    3.8085
p4hxrf    3.9470
jvccx2    4.0720
6m

In [10]:
df.groupby('participant').delay.describe().sort_values(by='min').head(10)

,count,mean,std,min,25%,50%,75%,max
participant,,,,,,,,
p475m7,67.0,4.776343,5.437048,0.007,2.3200,3.261,4.6540,31.745
283hmx,67.0,1.860537,1.759908,0.067,1.1805,1.594,1.8375,12.711
6pmv8x,67.0,1.589060,0.750829,0.082,1.1380,1.401,1.9555,4.247
p3x3wj,67.0,5.100567,14.024209,0.109,1.4525,2.012,3.3635,100.930
6fgx6j,67.0,2.861970,3.713779,0.144,1.3555,1.851,2.5745,23.261
xvxq7w,67.0,2.823642,1.545386,0.156,1.8930,2.501,3.4350,9.717
xx7p4w,67.0,1.976000,1.342354,0.183,1.2590,1.618,2.3500,7.144
9c8wwq,67.0,1.659045,1.137401,0.191,0.9655,1.271,2.2265,5.960
cxm3q7,67.0,10.715836,43.734768,0.195,1.1645,2.301,3.8920,353.096
